In [1]:
# !pip install tensorflow_data_validation[visualization]
# !pip install tensorflow-data-validation
!pip install --upgrade 'tensorflow_data_validation[visualization]'

ERROR: Invalid requirement: ''tensorflow_data_validation[visualization]''



In [2]:
import tensorflow_data_validation as tfdv
import pandas as pd

C:\Users\Simran\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## Read the training data

In [7]:
# 
df_train = pd.read_csv('data/train.tsv', sep='\t')
df_train.head()

,title,tags
0,How to draw a stacked dotplot in R?,['r']
1,mysql select all records where a datetime fiel...,"['php', 'mysql']"
2,How to terminate windows phone 8.1 app,['c#']
3,get current time in a specific country via jquery,"['javascript', 'jquery']"
4,Configuring Tomcat to Use SSL,['java']


In [5]:
df_train.describe()

,title,tags
count,100000,100000
unique,99984,7066
top,Conversion failed when converting date and/or ...,['java']
freq,3,7026


In [6]:
df_train.shape

(100000, 2)

## Read the validation data

In [11]:
df_val = pd.read_csv('data/validation.tsv', sep='\t')
df_val.head()

,title,tags
0,Why odbc_exec always fail?,"['php', 'sql']"
1,Access a base classes variable from within a c...,['javascript']
2,"Content-Type ""application/json"" not required i...","['ruby-on-rails', 'ruby']"
3,Sessions in Sinatra: Used to Pass Variable,"['ruby', 'session']"
4,"Getting error - type ""json"" does not exist - i...","['ruby-on-rails', 'ruby', 'json']"


In [12]:
df_val.describe()

,title,tags
count,30000,30000
unique,29996,3482
top,Object reference not set to an instance of an ...,['java']
freq,2,2044


In [13]:
df_val.shape

(30000, 2)

## Read the test data

In [8]:
df_test = pd.read_csv('data/test.tsv', sep='\t')
df_test.head()

,title
0,Warning: mysql_query() expects parameter 2 to ...
1,get click coordinates from <input type='image'...
2,How to implement cloud storage for media asset...
3,What is catcomplete in jQuery's autocomplete p...
4,Error building Android app with Cordova 3.1 CLI


## Use tfdv to compute statistics for the training set

In [15]:
train_stats = tfdv.generate_statistics_from_dataframe(df_train)

In [16]:
tfdv.visualize_statistics(train_stats)

## Use tfdv to compute statistics for the validation set

In [19]:
val_stats = tfdv.generate_statistics_from_dataframe(df_val)
tfdv.visualize_statistics(val_stats)

## Use tfdv to compute statistics for the training set

In [27]:
test_stats = tfdv.generate_statistics_from_dataframe(df_test)
tfdv.visualize_statistics(test_stats)

# Data validation

## Create schema

In [23]:
schema = tfdv.infer_schema(train_stats)

In [24]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'title',BYTES,required,,-
'tags',BYTES,required,,-


## Inspect anomalies in validation set

In [29]:
anomalies = tfdv.validate_statistics(val_stats, schema=schema)
tfdv.display_anomalies(anomalies)

## Inspect anomalies in test set

In [30]:
anomalies = tfdv.validate_statistics(test_stats, schema=schema)
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'tags',Column dropped,Column is completely missing


In [31]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('TESTING')

In [32]:
# Specify that tags column is not in TESTING environment.
tfdv.get_feature(schema, 'tags').not_in_environment.append('TESTING')

In [34]:
serving_anomalies_with_env = tfdv.validate_statistics(
        test_stats, schema, environment='TESTING') 
tfdv.display_anomalies(serving_anomalies_with_env)

In [37]:
tfdv.visualize_statistics(lhs_statistics=train_stats, rhs_statistics=val_stats, lhs_name='TRAIN', rhs_name='VAL')

In [38]:
tfdv.visualize_statistics(lhs_statistics=train_stats, rhs_statistics=test_stats, lhs_name='TRAIN', rhs_name='TEST')